# This is the first step in running corridor analysis for the Caribbean portion of the Blueprint

We are testing different things on the Virgin Islands / Puerto Rico corridor run. We don't have road crossing data here. We also don't go far into the deep marine, so our approach to marine vs inland runs won't be the same as in the base blueprint.

Created by Amy Keister, last run by Amy Keister on 9/6/2023. It took one minute to run.

In [51]:
import os
import arcpy
import numpy

In [52]:
import time
start = time.time()

In [53]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\G_VIPRresistance.gdb"

In [67]:
# define final outputs
Out = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ha_VIPR_LM_Input.gdb\VIPRResistance"

In [55]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [56]:
# define additional inputs
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"
lulc= r"F:\GIS_DATA\LanduseLandcover\Landfire\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"

# we didn't have any road crossing data for VIPR yet, but wanted to keep this code incase we leared of some during workshops
#RoadX= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\NCandFLWildlifeRoadCrossings.shp"
# I had to do this buffer outside of the model, because it would not buffer all point inside the model
#RoadX210buff= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\NCandFLWildlifeRoadCrossings_Buffer210m.shp"
#Road= r"F:\GIS_DATA\LanduseLandcover\WildlifeRoadCrossings\WildlifeRoadCrossingsNCFL.gdb\RoadsAroundNCandFLWildlifeRoadCrossings"

# Start Analysis

In [57]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [58]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\G_VIPRresistance.gdb


# Bring in Zonation results

In [60]:
# flip zonation results so that high is low and low is high to make resistance layer
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Minus(100, Zonation); out_raster.save("ZonationFlip")

In [ ]:
# take zonation times 100
# I only needed this on the workshop run where I didn't start with the combined zonation raster
#out_raster = arcpy.sa.Times("ZonationFlip", 100); out_raster.save("ZonationFlip100")

In [ ]:
# convert to integer
# I only needed this on the workshop run where I didn't start with the combined zonation raster
#out_raster = arcpy.sa.Int("ZonationFlip100"); out_raster.save("ZonationFlip100Int")

# Burn Landfire urban in to try to influence corridors to go around


In [61]:
# Make a raster where developed (low, medium, high, roads, or open space) is 0 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(lulc, 500, '', "Value IN (7296, 7297, 7298, 7299, 7300)"); out_raster.save("Dev")

In [63]:
# Combine urban with the zonation results, using a maximum value so that these urban areas are very hard to cross
# I only needed this on the workshop run where I didn't start with the combined zonation raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.CellStatistics(["ZonationFlip100Int", "Dev"], "MAXIMUM", 'DATA'); out_raster.save("ZonationFlip100IntUrb")

In [69]:
# Combine urban with the zonation results, using a maximum value so that these urban areas are very hard to cross
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics(["ZonationFlip", "Dev"], "MAXIMUM", 'DATA'); out_raster.save(Out)

In [68]:
# export
#arcpy.management.CopyRaster("ZonationFlipUrb", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [ ]:
end = time.time()
print(end - start)

# Below here is extra code not used this year, but that may be helpful in the future

## make known wildlife road crossings easy to move through
Right now I only have a few known road crossings in eastern North Carolina, but I'm looking for more

In [ ]:
# make a copy of the road crossing data because it is being stupid
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
#    arcpy.management.CopyFeatures(RoadX, "RoadX", '', None, None, None)

In [ ]:
# buffer road crossings by 210 meters

#This is maddening and bizarre, but when I run this in the model, it will only buffer
# a handfull of these points. I have no idea why, it works fine outside of the model.
#arcpy.analysis.Buffer("RoadX", "RoadXbuff", "210 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [ ]:
# buffer road crossings by 210 meters
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="MosaicInlandNoEst", snapRaster=SEraster, cellSize=SEraster):
#    arcpy.analysis.Buffer(RoadXa, "RoadXb", "210 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [ ]:
# add and calculate field to use to raster conversion
arcpy.management.CalculateField(RoadX210buff, "raster", 1, "PYTHON3", '', "SHORT")

In [ ]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster(RoadX210buff, "raster", "RoadXR", "CELL_CENTER", "NONE", SEraster)

In [ ]:
# now bring in roads around wildlife road crossings, we want to make these extra hard to move through in the 
# resistance raster
# buffer roads around crossings by 180 meters
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer(Road, "Road", "180 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [ ]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster("Road", "BUFF_DIST", "RoadR", "CELL_CENTER", "NONE", SEraster)

In [ ]:
# reclassify roads near wildlife road crossings and give a value of 500
out_raster = arcpy.sa.Reclassify("RoadR", "Value", "180 500", "DATA"); out_raster.save("RoadR2")

In [ ]:
# maybe also do a larger buffer and make it low but not zero resistance
# buffer road crossings by 450 meters
arcpy.analysis.Buffer(RoadX, "RoadX2", "900 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

In [ ]:
# erase road area from the larger buffered road crossing
arcpy.analysis.Erase("RoadX2", "Road", "RoadX2_Erase", None)

In [ ]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent="MosaicInlandNoEst"):
    arcpy.conversion.PolygonToRaster("RoadX2_Erase", "SymbolID", "RoadX2R", "CELL_CENTER", "NONE", SEraster)

In [ ]:
# reclassify larger buffer and give a value of 10
out_raster = arcpy.sa.Reclassify("RoadX2R", "Value", "0 10", "DATA"); out_raster.save("RoadX3R")

In [ ]:
# Combine roads with the mosaiced zonation results, using a maximum value so that these roads are very hard to cross
out_raster = arcpy.sa.CellStatistics(["MosaicInlandNoEstUrb", "RoadR2"], "MAXIMUM", 'DATA'); out_raster.save("MosaicInRoad")

In [ ]:
# Combine above raster with the road crossing data, using a minimum value so that the areas with road crossings are
# easier to cross
out_raster = arcpy.sa.CellStatistics(["MosaicInRoad","RoadX3R", "RoadXR"], "MINIMUM", 'DATA'); out_raster.save("MosaicInRoadX")

In [ ]:
# use times to remove the urban and road areas that fall outside extent
out_raster = arcpy.sa.Times("MosaicInRoadX", "InlandNearShoreBuffRNoEst"); out_raster.save("MosaicInRoadXa")

# bring in potential corridors
In the 2021 South Atlantic Blueprint, we used a new method where we use corridors from other projects to try to help keep corridors more stable. The idea is that we use our old corridors and corridors from other projects and use them to limit the resistance raster to try to get corridors to move through them.

Since we didn't have previous corridors for the whole Base Blueprint area, we didn't do that this year. We expect to pick this back up in 2023 base blueprint.

In [ ]:
# Make eastern wildway core a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(EWCore, "Id", "EWCore", 30)

In [ ]:
# Make eastern wildway corridor a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(EWCorr, "Id", "EWCorr", 30)

In [ ]:
# Make SA 2020 hub a raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    arcpy.conversion.FeatureToRaster(SA20Hub, "GRIDCODE", "SA20Hub", 30)

In [ ]:
# Select out top portion of SA 2020 linkagemapper corridor output
# I'm picking 500000 as a cutoff, It may be helpful to move that around in testing
# could try 1000000 or 1200000
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    out_raster = arcpy.sa.Con(SA20Cor, 1, None, "Value < 500000"); out_raster.save("SA20Cor")

In [ ]:
# combine potential cores/corridors
#out_raster = arcpy.sa.CellStatistics(["EWCore","EWCorr","SA20Hub", "SA20Cor"], "MAXIMUM", 'DATA'); out_raster.save("Prev")

In [ ]:
#reclassify 
#out_raster = arcpy.sa.Reclassify("Prev", "Value", "0 500 1;NODATA 0", "DATA"); out_raster.save("Prev1")

In [ ]:
# change NODATA to zero so expand will work
#out_raster = arcpy.sa.Reclassify("Prev", "Value", "1 1; 0 0; NODATA 0", "DATA"); out_raster.save("Top10zROW2")

In [ ]:
# expand to get rid of small gaps
#out_raster = arcpy.sa.Expand("Prev1", 2, 1, ''); out_raster.save("Prev2")

In [ ]:
# shrink to bring it back to size
#out_raster = arcpy.sa.Shrink("Prev2", 2, 1, ''); out_raster.save("Prev3")

In [ ]:
# change 0 to NoData
#out_raster = arcpy.sa.Reclassify("Prev3", "Value", "1 1;0 NODATA", "DATA"); out_raster.save("Prev4")

# limit resistance raster to areas around previous corridors

In [ ]:
# limit resistance raster to areas around previous corridors, this is the most dramatic option and does not allow
# linkagemapper to even consider areas that were not around previous corridors
#with arcpy.EnvManager(outputCoordinateSystem=sr, mask="InlandR", snapRaster=SAraster, cellSize=SAraster):
#    out_raster = arcpy.sa.ExtractByMask("MosaicInRoadX", "Prev4"); out_raster.save("ResistanceTest7")

# clip to test area

In [ ]:
# clip to a test area so I can see how it is working
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Pextent):
#    out_raster = arcpy.sa.Times("ResistanceTest7_90mBilinear", 1); out_raster.save("ResistanceTest7_90mBilinearTest")